In [1]:
import json
import datetime
from os.path import abspath
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

warehouse_location = abspath('/home/jovyan/work/spark-warehouse')

spark = SparkSession \
    .builder \
    .appName("shopee-category-search-extract") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("spark.debug.maxToStringFields",200) \
    .config("spark.sql.debug.maxToStringFields",2000) \
    .config("spark.executor.memory", "6g") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.extraJavaOptions","-Dio.netty.tryReflectionSetAccessible=true -Xms6114m") \
    .config("spark.driver.extraJavaOptions","-Dio.netty.tryReflectionSetAccessible=true -Xms6114m") \
    .enableHiveSupport() \
    .getOrCreate()

spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

spark.sql("select current_date").show()

print(spark.sparkContext._conf.getAll())

day=datetime.datetime.now().strftime("%Y-%m-%d")
print(day)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/27 11:11:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------------+
|current_date()|
+--------------+
|    2021-12-27|
+--------------+

[('spark.driver.host', 'ad1515b12d9c'), ('spark.executor.extraJavaOptions', '-Dio.netty.tryReflectionSetAccessible=true -Xms6114m'), ('spark.executor.id', 'driver'), ('spark.sql.debug.maxToStringFields', '2000'), ('spark.executor.memory', '6g'), ('spark.app.id', 'local-1640603469439'), ('spark.sql.catalogImplementation', 'hive'), ('spark.driver.memory', '6g'), ('spark.driver.port', '35215'), ('spark.rdd.compress', 'True'), ('spark.debug.maxToStringFields', '200'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.sql.warehouse.dir', 'file:/home/jovyan/work/spark-warehouse'), ('spark.app.startTime', '1640603468760'), ('spark.driver.extraJavaOptions', '-Dio.netty.tryReflectionSetAccessible=true -Xms6114m'), ('spark.app.name', 'shopee-category-search-extract'), ('spark.ui.showConsoleProgress', 'true')]

In [2]:
shopee_category_search_raw_df="/home/jovyan/work/spark-warehouse/ods.db/shopee_category_search_raw_df"
    
def write_to_hive(df):
    spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")
    df.write\
        .mode("overwrite")\
        .partitionBy("day")\
        .option("path",shopee_category_search_raw_df)\
        .format("parquet")\
        .saveAsTable("shopee_category_search_raw_df");

In [12]:
import shutil
import os

def read_file_into_df(file_path):
    df=spark.read.json(file_path)
    return df
        



#         旧方法：已经不适用
# def read_file_into_df(file, is_binary=False):
#     msg=file.readline()
#     json_str_arr=[]
#     # final_df=None
#     while msg is not None:
#         if is_binary:
#             msg=msg.decode("utf-8")
#         date_and_json=msg.split(" -json- ")
#         msg=file.readline()
#         if len(date_and_json)<=1:
#             break
#         json_obj=None
#         try:
#             json_obj=json.loads(date_and_json[1])
#         except:
#             print("json str might be wrong:\n",date_and_json[1])
#             continue

#         if json_obj['source']=='shopee':
#             if 'items' not in json_obj.keys() or len(json_obj['items']) <=0:
#                 print("this data has no items:",json_obj)
#                 continue
#             json_obj['request_time']=date_and_json[0]
#             json_obj['day']=date_and_json[0].split(" ")[0]
#             #减少体积
#             #TODO:
#             json_obj['search_tracking']='ignore'
#             json_obj['algorithm']='ignore'
            
#             json_str_arr.append(json.dumps(json_obj))

#     if len(json_str_arr)>0:
#         return json_str_arr
#     else:
#         print("no data.")
#         return None;
        


In [13]:
# import os
# import gzip

# gzip_dir="/home/jovyan/work/json/messages-archived/"

# arr = os.listdir(gzip_dir)

# df_list=[]

# if len(arr)>0:
#     for zipped_file in arr:
#         if 'messages.'+day in zipped_file:
#             print("about to read:", zipped_file)
#             with gzip.open(gzip_dir+zipped_file, 'r') as f:
#                 df=read_file_into_df(f,is_binary=True)
#                 if df is None:
#                     print("zipped_file returned none df:", zipped_file)
#                     continue
#                 print("zipped_file df size:", zipped_file, len(df))
#                 df_list.append(df)

In [14]:
import hashlib

def sha256sum(filename):
    h  = hashlib.sha256()
    b  = bytearray(128*1024)
    mv = memoryview(b)
    with open(filename, 'rb', buffering=0) as f:
        for n in iter(lambda : f.readinto(mv), 0):
            h.update(mv[:n])
    return h.hexdigest()

df=read_file_into_df("/home/jovyan/work/log-received/api_api_v4_search_search_items.json")

# df=spark.read.json("json/q_api_v4_search_search_items.json.processing")

print("df size:", df.count())
df.printSchema()
write_to_hive(df)
display(df.groupBy("day").count().toPandas())

# if df is not None:
#     df_final=df
#     if len(df_list)>0:
#         for zipped_df in df_list:
#             df_final+=zipped_df
            
#     df_final = spark.read.json(spark.sparkContext.parallelize(df_final))
#     write_to_hive(df_final)
#     spark.sql("""select day,count(1) 
#             from shopee_category_search_raw_df 
#             group by day order by day""").show()

json/messages.log size: 1632
root
 |-- adjust: struct (nullable = true)
 |    |-- count: long (nullable = true)
 |-- algorithm: string (nullable = true)
 |-- day: string (nullable = true)
 |-- disclaimer_infos: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- food_item_info: struct (nullable = true)
 |    |-- total_count: long (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- bff_item_tracking: string (nullable = true)
 |    |    |-- item_basic: struct (nullable = true)
 |    |    |    |-- add_on_deal_info: struct (nullable = true)
 |    |    |    |    |-- add_on_deal_id: long (nullable = true)
 |    |    |    |    |-- add_on_deal_label: string (nullable = true)
 |    |    |    |    |-- status: long (nullable = true)
 |    |    |    |    |-- sub_type: long (nullable = true)
 |    |    |    |-- badge_icon_type: long (nullable = true)
 |    |    |    |-- brand: string (nullable = true)
 |    | 